In [3]:
from gensim.models import KeyedVectors
import pandas as pd
from collections import Counter
import nltk
import re
import requests
import pickle
import json
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import textwrap               # ➊ NEW

from tensorflow.python.distribute.strategy_combinations import parameter_server_strategy_1worker_2ps_1gpu



2025-07-10 22:48:09.657351: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 22:48:09.670665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752180489.686442 1059646 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752180489.690853 1059646 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752180489.702602 1059646 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
metadata_table_long = pd.read_json("../data/metadata_table_long.json")
metadata_table_long.head(5)

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Original,...,ids,id,date_min,date_max,filename,file_year,sents_n,tokens_n,disciplines_list,science_class
0,"Achrelius, Daniel",Scientiarum magnes recitatus publice anno 1690...,None,1690,[Turku],Wall,17th century,Oration,"Mathematics, Astronomy/Astrology/Cosmography, ...",Scientiarum magnes(Google Books),...,[705665],705665,1690.0,1690.0,"Achrelius,_Daniel_-_Scientiarum_magnes__Turku_...",1690.0,630,8839,"[Mathematics, Astronomy/Astrology/Cosmography,...",mixture
1,"Acidalius, Valens","Ad Iordanum Brunum Nolanum, Italum","Poematum Iani Lernutii, Iani Gulielmi, Valenti...",1603,"Liegnitz, Wrocław","Albert, David",17th century,Panegyric poem,Astronomy/Astrology/Cosmography,Ad Iordanum Brunum (1603)(CAMENA)Ad Iordanum B...,...,[801745],801745,1603.0,1603.0,Janus_Lernutius_et_al__-_Poemata__Liegnitz_160...,1603.0,6527,82445,[Astronomy/Astrology/Cosmography],exact_sciences
2,"Acosta, José de",De natura novi orbis libri duo et De promulgat...,None,1589,Salamanca,Guillelum Foquel,16th century,Monograph,"Astronomy/Astrology/Cosmography, Geography/Car...",De natura novi orbis(Biodiversity Heritage Lib...,...,[713323],713323,1589.0,1589.0,"Acosta,_José_de_-_De_natura_novi_orbis__Salama...",1589.0,8905,170205,"[Astronomy/Astrology/Cosmography, Geography/Ca...",exact_sciences
3,"Adam, Melchior","Vitae Germanorum medicorum, qui saeculo superi...",None,1620,Heidelberg,"Rosa, Geyder",17th century,Biography,Medicine,Vitae Germanorum medicorum(MDZ)Alternative lin...,...,[693148],693148,1620.0,1620.0,"Adam,_Melchior_-_Vitae_Germanorum_medicorum__H...",1620.0,17961,193272,[Medicine],life_sciences
4,"Addison, Joseph",Ad insignissimum virum dominum Thomam Burnettu...,"Examen poeticum duplex, sive, Musarum anglican...",1698,London,Richard Wellington I.,17th century,Panegyric poem,Meteorology/Earth sciences,Ad Burnettum sacrae theoriae telluris auctorem...,...,[769230],769230,1698.0,1698.0,Examen_poeticum_duplex__London_1698_pdf.txt,1698.0,3533,47878,[Meteorology/Earth sciences],exact_sciences


## Subcorpora word counts

In [3]:
disciplines_list = ['Medicine',
 'Astronomy/Astrology/Cosmography',
 'Biology',
 'Mathematics',
 'Meteorology/Earth sciences',
 'Physics',
 'Geography/Cartography',
 'Alchemy/Chemistry']

In [4]:
subcorpus_counts = []
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"

periods = [(1501, 1550),
                      (1551, 1600),
                      (1601, 1650),
                      (1651, 1700)]
periods_str = [str(per[0]) + "-" + str(per[1]) for per in periods]
for per_tup, per_str in zip(periods, periods_str):
    sents_n = 0
    tokens_n = 0
    ids = metadata_table_long[metadata_table_long["file_year"].between(per_tup[0], per_tup[1])]["id"]
    for id in ids:
        f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
        sents_n += len(f_sents_data)
        for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
            tokens_n += len(sent_data)
    subcorpus_counts.append({"subcorpus": per_str, "sents_n": sents_n, "tokens_n": tokens_n})

for discipline in disciplines_list:
    sents_n = 0
    tokens_n = 0
    ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
    for id in ids:
        f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
        sents_n += len(f_sents_data)
        for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
            tokens_n += len(sent_data)
    subcorpus_counts.append({"subcorpus": discipline, "sents_n": sents_n, "tokens_n": tokens_n})

source_path = "/srv/data/tome/tome-corpus/sents_data_id_jsons_v3-0/"
emlap_metadata = pd.read_csv("/srv/data/tome/tome-corpus/emlap_metadata.csv", sep=";")
ids = emlap_metadata["No."]

sents_n = 0
tokens_n = 0
for id in ids:
    f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
    sents_n += len(f_sents_data)
    for (doc_id, sent_id, sent_text, sent_data) in f_sents_data:
        tokens_n += len(sent_data)
subcorpus_counts.append({"subcorpus": "EMLAP", "sents_n": sents_n, "tokens_n": tokens_n})

In [5]:
subcorpus_counts_df = pd.DataFrame(subcorpus_counts)
subcorpus_counts_df

,subcorpus,sents_n,tokens_n
0,1501-1550,1018967,12779473
1,1551-1600,2679563,35876603
2,1601-1650,1423173,17932748
3,1651-1700,2019943,27211868
4,Medicine,3210357,36896246
5,Astronomy/Astrology/Cosmography,2095377,29907378
6,Biology,2032483,25073799
7,Mathematics,1415929,19814766
8,Meteorology/Earth sciences,1215475,14833414
9,Physics,1256840,16344953


In [6]:
subcorpus_counts_df.to_csv("../data/subcorpus_counts.csv", index=False)

## Subcorpora word counts plot

In [ ]:
colors = ["green"] * len(disciplines_list) + ["blue"] * len(periods_str) + ["red"]

In [ ]:
# 0. --- data shortcuts ---
df      = subcorpus_counts_df     # alias for brevity
heights = df['tokens_n']
colors  = colors                  # comes from your earlier code

# 1. --- wrap long labels (12 chars/line) ➋ NEW ---
wrapped_labels = [
    textwrap.fill(lbl, width=12, break_long_words=False, break_on_hyphens=False)
    for lbl in df['subcorpus']
]

# 2. --- bar plot ---
x = np.arange(len(df))           # ➌ integer positions
fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.bar(x, heights, color=colors)

# 3. --- value labels above bars (unchanged) ---
for bar, value in zip(bars, heights):
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        height + heights.max() * 0.01,
        f'{value/1e6:.1f}M',
        ha='center', va='bottom', fontsize=10
    )

# 4. --- axes labels & title (unchanged) ---
ax.set_xlabel('Subcorpus')
ax.set_ylabel('Number of Tokens')
ax.set_title('Token Counts by Subcorpus')

# 5. --- set the new x-tick labels ➍ NEW ---
ax.set_xticks(x)
ax.set_xticklabels(
    wrapped_labels,
    rotation=60,            # steeper rotation
    ha='right',
    rotation_mode='anchor'
)

# 6. --- y-axis formatter (unchanged) ---
ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y, _: f'{y/1e6:.1f}M')
)

# 7. --- legend (unchanged) ---
legend_elements = [
    Patch(facecolor='green', label='Disciplines'),
    Patch(facecolor='blue',  label='Time Periods'),
    Patch(facecolor='red',   label='EMLAP')
]
ax.legend(handles=legend_elements, loc='upper right')

# 8. --- layout tweak ---
fig.subplots_adjust(bottom=0.20)   # a bit more space for wrapped labels
plt.tight_layout()


In [ ]:
fig.savefig("../figures/subcorpus_counts.png")

## LiLa embeddings

In [7]:
lasla = KeyedVectors.load_word2vec_format("/srv/data/lila/allLASLA-lemmi-fast-100-SKIP-win5-min5.vec")
lasla.most_similar("seminarium")

[('carnarium', 0.8714821934700012),
 ('sudarium', 0.8584282398223877),
 ('uinarius', 0.8317396640777588),
 ('torcularium', 0.8269298672676086),
 ('armarium', 0.8191075325012207),
 ('milliarium', 0.8161847591400146),
 ('mortarium', 0.8139162063598633),
 ('asinarius', 0.7973706722259521),
 ('pomarium', 0.7952086329460144),
 ('olearius', 0.7895564436912537)]

In [8]:
lasla.most_similar("semen", topn=20)

[('sementis', 0.7769805788993835),
 ('germen', 0.65008145570755),
 ('legumen', 0.6492694020271301),
 ('materies', 0.62647545337677),
 ('sulcus', 0.6210276484489441),
 ('stercoro', 0.6130126118659973),
 ('genitalis', 0.6120802760124207),
 ('sumen', 0.6056757569313049),
 ('animans', 0.5975176095962524),
 ('elementa', 0.5954310297966003),
 ('seges', 0.5945314764976501),
 ('primordium', 0.5920283198356628),
 ('bipalium', 0.5892609357833862),
 ('omnigenus', 0.5877910256385803),
 ('ablaqueo', 0.5792651772499084),
 ('renascor', 0.573520839214325),
 ('enascor', 0.5693485736846924),
 ('abdomen', 0.5686165690422058),
 ('priuus', 0.5666219592094421),
 ('augmen', 0.5655485391616821)]

In [9]:
lasla.index_to_key[:10]

['</s>', 'sum', 'et', 'qui', 'in', 'non', 'hic', 'is', 'ut', 'ego']

In [10]:
operamaiora = KeyedVectors.load_word2vec_format("/srv/data/lila/opera-maiora-lemmas_skip_100.vec")
operamaiora.most_similar("anima")

[('intellectiuus', 0.7068129181861877),
 ('separo', 0.7005686163902283),
 ('sensifico', 0.6973943114280701),
 ('sensitiuus', 0.6771703362464905),
 ('corpus', 0.663750171661377),
 ('uegetabilis', 0.6309762597084045),
 ('unio', 0.6247175335884094),
 ('rationalis', 0.623648464679718),
 ('uegetatiuus', 0.618419885635376),
 ('corporeus', 0.60992830991745)]

In [11]:
operamaiora.most_similar("semen")

[('formatiuus', 0.7347741723060608),
 ('decidocaedo', 0.7292525768280029),
 ('sementis', 0.7260836362838745),
 ('decisio', 0.718953549861908),
 ('proauus', 0.708860456943512),
 ('menstruus', 0.7087300419807434),
 ('fetus', 0.7033671736717224),
 ('incubus', 0.6823228597640991),
 ('seminalis', 0.6802768111228943),
 ('spumosus', 0.6695918440818787)]

In [12]:
operamaiora_vocab = operamaiora.index_to_key
len(operamaiora_vocab)

6724

In [13]:
lasla_vocab = lasla.index_to_key
len(lasla_vocab)

11327

In [14]:
vocab_overlap = list(set(lasla_vocab) & set(operamaiora_vocab))
len(vocab_overlap)

3950

## Build vocabulary data

In [15]:
source_path = "/srv/data/tome/noscemus/sents_data_jsons/"

In [16]:
id = os.listdir(source_path)[0]

In [17]:
f_sents_data = json.load(open(source_path + str(id), "rb"))
f_sents_data[:3]

[['604894',
  0,
  'Noua Acta Heluetica Physico-mathematico- Anatomico-botanico- Medica Tabulis Aeneis Illustrata Et In Usus Publicos Exarata.',
  [['Noua', 'nouus', 'ADJ', [0, 4]],
   ['Acta', '', 'VERB', [5, 9]],
   ['Heluetica', 'helueticus', 'ADJ', [10, 19]],
   ['Physico', 'physicus', 'PROPN', [20, 27]],
   ['-', '-', 'PUNCT', [27, 28]],
   ['mathematico-', '', 'ADJ', [28, 40]],
   ['Anatomico', 'anatomicus', 'PROPN', [41, 50]],
   ['-', '-', 'PUNCT', [50, 51]],
   ['botanico-', '', 'PRON', [51, 60]],
   ['Medica', 'medica', 'PROPN', [61, 67]],
   ['Tabulis', 'tabula', 'NOUN', [68, 75]],
   ['Aeneis', 'aeneus', 'ADJ', [76, 82]],
   ['Illustrata', 'illustro', 'VERB', [83, 93]],
   ['Et', 'et', 'CCONJ', [94, 96]],
   ['In', 'in', 'ADP', [97, 99]],
   ['Usus', 'usus', 'NOUN', [100, 104]],
   ['Publicos', 'publicus', 'PROPN', [105, 113]],
   ['Exarata', 'exaro', 'VERB', [114, 121]],
   ['.', '.', 'PUNCT', [121, 122]]]],
 ['604894',
  1,
  'Uolumen I. Basileae, Typis & Sumptibus Joanni

In [18]:
def get_filtered_counts_by_ids(ids, lowercase=True):
    vocab_counter = Counter() # initiate an empty counter of words - we will feed it iterarively on the go by all words we meet
    for id in ids: # for each work ID from our subset of IDs
        # based on the ID, open the file containing the lemmatized sentences
        f_sents_data = json.load(open(source_path + str(id) + ".json", "rb"))
        sents_n = len(f_sents_data)
        tokens_n = 0
        lemmata = []
        for sent_data in f_sents_data:
            tokens_n += len(sent_data[3])
            sent_lemmata = [t[1] for t in sent_data[3] if t[2] in ["NOUN", "VERB", "ADJ", "PROPN"]]
            sent_lemmata = [re.sub(r"\W*|\d*", "", t) for t in sent_lemmata]
            sent_lemmata = [l for l in sent_lemmata if len(l) > 1]
            lemmata.extend(sent_lemmata)
        if lowercase:
            lemmata = [l.lower() for l in lemmata]
        #overview_dict[filename.partition(".")[0]] = {"sents_n" : sents_n, "tokens_n" : tokens_n} 
        word_counts = dict(nltk.FreqDist(lemmata).most_common())
        vocab_counter.update(word_counts)
    return vocab_counter

In [19]:
ids = metadata_table_long[metadata_table_long["file_year"].between(1600, 1610)]["id"]
vocab = get_filtered_counts_by_ids(ids)

In [20]:
list(vocab.items())[:100]

[('amor', 436),
 ('animus', 907),
 ('do', 2857),
 ('possum', 5880),
 ('habeo', 6613),
 ('google', 697),
 ('uideo', 4251),
 ('ualeo', 441),
 ('facio', 5891),
 ('uita', 1278),
 ('dico', 6654),
 ('deus', 1036),
 ('fero', 1987),
 ('lux', 1082),
 ('oculus', 2649),
 ('nomen', 2136),
 ('dies', 5150),
 ('mens', 601),
 ('os', 6732),
 ('musa', 193),
 ('acidalius', 102),
 ('bonus', 1010),
 ('uis', 2453),
 ('fides', 612),
 ('janus', 98),
 ('uolo', 1823),
 ('nouus', 952),
 ('uiuo', 677),
 ('dolor', 1055),
 ('pectus', 599),
 ('manus', 1347),
 ('ingenium', 469),
 ('tempus', 4281),
 ('homo', 2138),
 ('amicus', 413),
 ('magnus', 3618),
 ('carmen', 185),
 ('cura', 397),
 ('orbis', 1154),
 ('ago', 1172),
 ('felix', 448),
 ('flos', 527),
 ('uenio', 1317),
 ('amo', 274),
 ('diuus', 159),
 ('cor', 2420),
 ('mors', 719),
 ('caput', 5098),
 ('eo', 368),
 ('numen', 111),
 ('annus', 5028),
 ('spes', 229),
 ('capio', 587),
 ('dignus', 426),
 ('credo', 611),
 ('puer', 442),
 ('res', 3639),
 ('ars', 1282),
 ('hono

In [21]:
# generate vocabularies for each half-century under scrutiny
periods = [(1501, 1550),
           (1551, 1600),
           (1601, 1650),
           (1651, 1700)]

periods_vocabs = []
full_vocabs = []
for per in periods:
    ids = metadata_table_long[metadata_table_long["file_year"].between(per[0], per[1])]["id"] # use the metadata table to extract IDs of all works falling into this period
    vocab = get_filtered_counts_by_ids(ids)
    periods_vocabs.append(vocab)

disciplines_list = ['Medicine',
 'Astronomy/Astrology/Cosmography',
 'Biology',
 'Mathematics',
 'Meteorology/Earth sciences',
 'Physics',
 'Geography/Cartography',
 'Alchemy/Chemistry']

disciplines_vocabs = []
for discipline in disciplines_list:
    ids = metadata_table_long[(metadata_table_long["file_year"].between(1501,1700)) & (metadata_table_long["disciplines_list"].apply(lambda x: discipline in x))]["id"]
    vocab = get_filtered_counts_by_ids(ids)
    disciplines_vocabs.append(vocab)

In [22]:
source_path = "/srv/data/tome/tome-corpus/sents_data_id_jsons_v3-0/"
emlap_metadata = pd.read_csv("/srv/data/tome/tome-corpus/emlap_metadata.csv", sep=";")
ids = emlap_metadata["No."]
emlap_vocab = [get_filtered_counts_by_ids(ids)]


In [23]:
list(periods_vocabs[0].items())[:10]

[('aqua', 11747),
 ('terra', 14371),
 ('lapis', 4154),
 ('metallum', 1984),
 ('color', 6554),
 ('res', 29069),
 ('dico', 43255),
 ('possum', 25029),
 ('facio', 31814),
 ('habeo', 31690)]

In [24]:
# only 2,000 most common words for each period...
topn = 2000
vocabs_topn_lists =[list(vocab.keys())[:topn] for vocab in periods_vocabs + disciplines_vocabs + emlap_vocab]

In [25]:
# build one list of words, including all from these four sets, i.e. their union
vocab_union = set.union(*[set(vocab) for vocab in vocabs_topn_lists])
len(vocab_union)

6689

In [26]:
# add there also the words appearing both in lasla and operamaiora
vocab_union = set.union(*[set(vocab) for vocab in vocabs_topn_lists] + [set(vocab_overlap)])
len(vocab_union)

8121

In [27]:
vocab_union_df = pd.DataFrame(vocab_union, columns = ["word"])
vocab_union_df

,word
0,laurus
1,rosa
2,irrisor
3,uas
4,super
...,...
8116,ludouicus
8117,distringo
8118,perscrutor
8119,arbitrium


In [28]:
# add the counts of these words from the vocabularies
def find_in_vocab(word, vocab):
    try:
        return vocab[word]
    except:
        return 0
for vocab, per in zip(periods_vocabs + disciplines_vocabs + emlap_vocab, periods_str + disciplines_list + ["EMLAP"]):
    vocab_union_df[per] = vocab_union_df["word"].apply(lambda x: find_in_vocab(x, vocab))

In [29]:
vocab_union_df.head(5)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP
0,laurus,803,1164,695,596,2077,524,1891,319,577,395,497,433,161
1,rosa,900,1979,1389,1017,3050,970,2428,523,559,647,556,448,742
2,irrisor,6,20,13,20,7,34,5,24,18,19,6,3,3
3,uas,2394,6381,3016,5832,8846,1694,5619,2362,3103,3942,1435,2609,4246
4,super,58,82,52,94,86,151,51,58,43,56,25,12,5


In [30]:
# calculate the mean value for sorting
vocab_union_df["mean"] = vocab_union_df[vocab_union_df.columns[1:]].mean(axis=1)
vocab_union_df.sort_values("mean", ascending=False, inplace=True)
vocab_union_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
2913,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,47600,39787,12754,14098,58211.692308
2835,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,50798,46481,29327,4593,50532.692308
6447,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,58081,27901,14528,14367,48999.384615
5187,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,40475,27720,11720,8377,46153.846154
5339,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,38497,32771,10723,9810,46132.000000
7784,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,40653,23879,11693,16068,45480.153846
220,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,44153,29452,13842,8935,40963.769231
5932,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,36674,20706,9607,6047,34549.384615
2804,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,27829,31499,7668,3810,33177.153846
1231,corpus,16979,63171,22871,31938,71457,42909,27375,19867,24931,32049,11300,8640,12498,29691.153846


In [31]:
vocab_union_df.tail(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
6704,necnon,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7990,frustra,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1414,etiam,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7644,quoquomodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7626,etiamsi,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7226,paene,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7220,quandoque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1093,qualiscumque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
735,tantummodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3336,quicumque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [32]:
# Filter only for words appearing at least 5 times in each of the subcorpora
#filtered_vocab_df = vocab_union_df[
#    (vocab_union_df['1501-1550'] >= 5) &
#    (vocab_union_df['1551-1600'] >= 5) &
#    (vocab_union_df['1601-1650'] >= 5) &
#    (vocab_union_df['1651-1700'] >= 5)
#    ]

In [33]:
filtered_vocab_df = vocab_union_df.copy()

In [34]:
filtered_vocab_df.head(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
2913,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,47600,39787,12754,14098,58211.692308
2835,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,50798,46481,29327,4593,50532.692308
6447,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,58081,27901,14528,14367,48999.384615
5187,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,40475,27720,11720,8377,46153.846154
5339,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,38497,32771,10723,9810,46132.000000
7784,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,40653,23879,11693,16068,45480.153846
220,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,44153,29452,13842,8935,40963.769231
5932,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,36674,20706,9607,6047,34549.384615
2804,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,27829,31499,7668,3810,33177.153846
1231,corpus,16979,63171,22871,31938,71457,42909,27375,19867,24931,32049,11300,8640,12498,29691.153846


In [35]:
filtered_vocab_df.tail(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean
6704,necnon,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7990,frustra,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1414,etiam,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7644,quoquomodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7626,etiamsi,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7226,paene,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7220,quandoque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1093,qualiscumque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
735,tantummodo,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3336,quicumque,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [36]:
filtered_vocab_df["in_lila_embeddings"] = filtered_vocab_df["word"].apply(lambda x: x in vocab_overlap)

In [37]:
filtered_vocab_df["in_lasla"] = filtered_vocab_df["word"].apply(lambda x: x in lasla_vocab)
filtered_vocab_df["in_operamaiora"] = filtered_vocab_df["word"].apply(lambda x: x in operamaiora_vocab)

In [38]:
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,Physics,Geography/Cartography,Alchemy/Chemistry,EMLAP,mean,in_lila_embeddings,in_lasla,in_operamaiora
2913,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,47600,39787,12754,14098,58211.692308,True,True,True
2835,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,50798,46481,29327,4593,50532.692308,True,True,True
6447,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,58081,27901,14528,14367,48999.384615,True,True,True
5187,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,40475,27720,11720,8377,46153.846154,True,True,True
5339,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,38497,32771,10723,9810,46132.000000,True,True,True
7784,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,40653,23879,11693,16068,45480.153846,True,True,True
220,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,44153,29452,13842,8935,40963.769231,True,True,True
5932,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,36674,20706,9607,6047,34549.384615,True,True,True
2804,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,27829,31499,7668,3810,33177.153846,True,True,True
1231,corpus,16979,63171,22871,31938,71457,42909,27375,19867,24931,32049,11300,8640,12498,29691.153846,True,True,True


In [39]:
filtered_vocab_df["in_lasla"].sum()

5298

In [40]:
filtered_vocab_df["in_operamaiora"].sum()

4349

In [41]:
filtered_vocab_df["in_lila_embeddings"].sum()


3950

## Add relative frequencies

In [43]:
subcorpus_counts_df

,subcorpus,sents_n,tokens_n
0,1501-1550,1018967,12779473
1,1551-1600,2679563,35876603
2,1601-1650,1423173,17932748
3,1651-1700,2019943,27211868
4,Medicine,3210357,36896246
5,Astronomy/Astrology/Cosmography,2095377,29907378
6,Biology,2032483,25073799
7,Mathematics,1415929,19814766
8,Meteorology/Earth sciences,1215475,14833414
9,Physics,1256840,16344953


In [55]:
for col, total in zip(filtered_vocab_df.columns[1:14], subcorpus_counts_df["tokens_n"]):
    filtered_vocab_df[col + "_freq"] = filtered_vocab_df[col].apply(lambda x: x/total)


In [56]:
filtered_vocab_df.head(5)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,1651-1700_freq,Medicine_freq,Astronomy/Astrology/Cosmography_freq,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq
2913,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.002654,0.002901,0.002718,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034
2835,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.001951,0.002651,0.002199,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314
6447,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002824,0.002204,0.002505,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110
5187,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.001925,0.002586,0.002206,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397
5339,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002196,0.002213,0.002111,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807


In [57]:
filtered_vocab_df.columns[18:]

Index(['1501-1550_freq', '1551-1600_freq', '1601-1650_freq', '1651-1700_freq',
       'Medicine_freq', 'Astronomy/Astrology/Cosmography_freq', 'Biology_freq',
       'Mathematics_freq', 'Meteorology/Earth sciences_freq', 'Physics_freq',
       'Geography/Cartography_freq', 'Alchemy/Chemistry_freq', 'EMLAP_freq'],
      dtype='object')

In [58]:
filtered_vocab_df["mean_freq"] = filtered_vocab_df[filtered_vocab_df.columns[18:]
].mean(axis=1)
filtered_vocab_df.sort_values("mean_freq", ascending=False, inplace=True)
filtered_vocab_df.head(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Medicine_freq,Astronomy/Astrology/Cosmography_freq,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq
2913,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.002901,0.002718,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921
2835,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002651,0.002199,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680
6447,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002204,0.002505,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563
7784,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.002616,0.002105,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367
5339,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002213,0.002111,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310
5187,pars,33225,93059,44337,52385,95410,65986,42118,53361,31827,...,0.002586,0.002206,0.001680,0.002693,0.002146,0.002476,0.001962,0.001895,0.002397,0.002279
220,res,29069,70669,28291,51565,74400,62681,42267,36572,40633,...,0.002016,0.002096,0.001686,0.001846,0.002739,0.002701,0.002085,0.002239,0.002556,0.002129
5932,uideo,21536,59099,26372,50503,61322,53447,47212,28471,28146,...,0.001662,0.001787,0.001883,0.001437,0.001897,0.002244,0.001466,0.001554,0.001730,0.001717
2804,locus,23102,56782,31421,43680,50814,57201,40768,31263,25466,...,0.001377,0.001913,0.001626,0.001578,0.001717,0.001703,0.002230,0.001240,0.001090,0.001632
3911,aqua,11747,41231,20765,34038,42778,18714,30884,14948,28253,...,0.001159,0.000626,0.001232,0.000754,0.001905,0.001437,0.001160,0.001966,0.005758,0.001575


In [62]:
filtered_vocab_df.head(10).round(6).to_csv("../data/filtered_vocab_df_top10.csv")

In [69]:
filtered_vocab_df.columns

Index(['word', '1501-1550', '1551-1600', '1601-1650', '1651-1700', 'Medicine',
       'Astronomy/Astrology/Cosmography', 'Biology', 'Mathematics',
       'Meteorology/Earth sciences', 'Physics', 'Geography/Cartography',
       'Alchemy/Chemistry', 'EMLAP', 'mean', 'in_lila_embeddings', 'in_lasla',
       'in_operamaiora', '1501-1550_freq', '1551-1600_freq', '1601-1650_freq',
       '1651-1700_freq', 'Medicine_freq',
       'Astronomy/Astrology/Cosmography_freq', 'Biology_freq',
       'Mathematics_freq', 'Meteorology/Earth sciences_freq', 'Physics_freq',
       'Geography/Cartography_freq', 'Alchemy/Chemistry_freq', 'EMLAP_freq',
       'mean_freq'],
      dtype='object')

In [70]:
filtered_vocab_df["shared"] = filtered_vocab_df[['1501-1550', '1551-1600', '1601-1650', '1651-1700', 'Medicine', 'Astronomy/Astrology/Cosmography', 'Biology', 'Mathematics', 'Meteorology/Earth sciences', 'Physics', 'Geography/Cartography', 'Alchemy/Chemistry', 'EMLAP']].ge(10).all(axis=1)

In [72]:
filtered_vocab_df["shared"].sum()

4804

## Remove remaining wrong words and save

In [73]:
filtered_vocab_df = filtered_vocab_df[filtered_vocab_df["word"].apply(lambda x: x not in ["google", "digitized", "by"])]

In [74]:
filtered_vocab_df.to_json("../data/filtered_vocab_df.json")

In [ ]:
!cp ../data/filtered_vocab_df.json /srv/data/tome/noscemus/filtered_vocab_df.json

## Explorations for article

In [86]:
len(filtered_vocab_df)

8118

In [85]:
subcorpus_counts_df["vectors_tokens"] = subcorpus_counts_df.apply(lambda row: filtered_vocab_df[filtered_vocab_df[row["subcorpus"]]>= 10][row["subcorpus"]].sum(), axis=1)
subcorpus_counts_df["vectors_coverage"] = subcorpus_counts_df["vectors_tokens"] / subcorpus_counts_df["tokens_n"]
subcorpus_counts_df

,subcorpus,sents_n,tokens_n,vectors_coverage,vectors_tokens
0,1501-1550,1018967,12779473,0.317445,4056786
1,1551-1600,2679563,35876603,0.295416,10598514
2,1601-1650,1423173,17932748,0.299126,5364155
3,1651-1700,2019943,27211868,0.288327,7845909
4,Medicine,3210357,36896246,0.299855,11063539
5,Astronomy/Astrology/Cosmography,2095377,29907378,0.289684,8663700
6,Biology,2032483,25073799,0.305500,7660058
7,Mathematics,1415929,19814766,0.281457,5577001
8,Meteorology/Earth sciences,1215475,14833414,0.315167,4675008
9,Physics,1256840,16344953,0.305876,4999531


In [83]:
subcorpus_counts_df

,subcorpus,sents_n,tokens_n,vectors_coverage
0,1501-1550,1018967,12779473,4056786
1,1551-1600,2679563,35876603,10598514
2,1601-1650,1423173,17932748,5364155
3,1651-1700,2019943,27211868,7845909
4,Medicine,3210357,36896246,11063539
5,Astronomy/Astrology/Cosmography,2095377,29907378,8663700
6,Biology,2032483,25073799,7660058
7,Mathematics,1415929,19814766,5577001
8,Meteorology/Earth sciences,1215475,14833414,4675008
9,Physics,1256840,16344953,4999531


## Add simple english translation

In [9]:
filtered_vocnab_df = pd.read_json("../data/filtered_vocab_df.json")

In [11]:
filtered_vocab_df.head(5)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Astronomy/Astrology/Cosmography_freq,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared
2913,dico,43255,122549,42457,72224,107036,81298,76957,52414,44323,...,0.002718,0.003069,0.002645,0.002988,0.002912,0.002816,0.002063,0.004034,0.002921,True
2835,liber,39559,65900,38496,53100,97802,65756,65230,49280,50603,...,0.002199,0.002602,0.002487,0.003411,0.003108,0.003290,0.004743,0.001314,0.002680,True
6447,possum,25029,82700,38391,76841,81305,74924,50255,50826,41844,...,0.002505,0.002004,0.002565,0.002821,0.003553,0.001975,0.002350,0.004110,0.002563,True
7784,facio,31814,98524,36944,51438,96530,62963,42272,45704,32760,...,0.002105,0.001686,0.002307,0.002209,0.002487,0.001690,0.001891,0.004597,0.002367,True
5339,habeo,31690,89199,39443,59770,81643,63141,61273,48571,33185,...,0.002111,0.002444,0.002451,0.002237,0.002355,0.002320,0.001734,0.002807,0.002310,True


In [1]:
# simple latin API:
# https://www.latin-is-simple.com/api/

In [4]:
target = "equus"
base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
resp_json = requests.get(base_url.format(target)).json()
resp_json

[{'id': 63,
  'intern_type': 'noun',
  'short_name': 'equus',
  'full_name': 'equus, equi [m.] O',
  'type': {'name': 'noun', 'label': 'Noun'},
  'translations_unstructured': {'en': 'horse', 'de': 'Pferd'},
  'url': 'https://www.latin-is-simple.com/en/vocabulary/noun/63/'}]

In [5]:
resp_json[0]["translations_unstructured"]["en"]

'horse'

In [6]:
def simple_translation(target):
    try:
        base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
        resp_json = requests.get(base_url.format(target)).json()
        transl = resp_json[0]["translations_unstructured"]["en"]
        if transl == "still in translation":
            transl = ""
        return transl
    except:
        return ""

In [13]:
import concurrent.futures

def get_translations_parallel(df, col_name="word", n_workers=30):
    words = df[col_name].tolist()
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_workers) as executor:
        results = list(executor.map(simple_translation, words))
    return results

In [12]:
len(filtered_vocab_df)

8118

In [14]:
%%time
filtered_vocab_df["transl"] = get_translations_parallel(filtered_vocab_df)

CPU times: user 38.1 s, sys: 3.23 s, total: 41.3 s
Wall time: 1min 45s


In [16]:
filtered_vocab_df.sample(10)

,word,1501-1550,1551-1600,1601-1650,1651-1700,Medicine,Astronomy/Astrology/Cosmography,Biology,Mathematics,Meteorology/Earth sciences,...,Biology_freq,Mathematics_freq,Meteorology/Earth sciences_freq,Physics_freq,Geography/Cartography_freq,Alchemy/Chemistry_freq,EMLAP_freq,mean_freq,shared,transl
6631,locito,26,21,14,34,18,33,7,33,11,...,2.792000e-07,1.665400e-06,7.416000e-07,2.202500e-06,3.539000e-07,1.617000e-07,0.000000e+00,8.958000e-07,False,
6615,totalis,127,455,471,1138,380,1415,156,596,174,...,6.221600e-06,3.007860e-05,1.173030e-05,3.597440e-05,1.592650e-05,1.342340e-05,1.716630e-05,2.144910e-05,True,"total, entire"
7619,adnitor,34,58,24,68,58,55,48,36,40,...,1.914300e-06,1.816800e-06,2.696600e-06,3.365000e-06,2.335900e-06,1.617300e-06,5.722000e-07,1.988000e-06,False,"lean/rest upon, support oneself, (w/genibus) k..."
7371,penetro,709,2295,1370,1963,3021,1556,1471,802,1449,...,5.866680e-05,4.047490e-05,9.768490e-05,8.296140e-05,5.896350e-05,1.191936e-04,1.951241e-04,8.115060e-05,True,"enter, penetrate"
5340,trucido,106,194,101,151,105,150,146,62,86,...,5.822800e-06,3.129000e-06,5.797700e-06,2.875500e-06,2.137690e-05,2.749400e-06,4.005500e-06,6.038600e-06,True,"slaughter, butcher, massacre"
5628,cognatus,270,1285,405,337,813,710,976,284,537,...,3.892510e-05,1.433270e-05,3.620210e-05,2.386060e-05,2.972950e-05,2.911110e-05,1.544970e-05,2.502300e-05,True,"related, related by birth/position, kindred, s..."
3152,praemia,24,16,3,5,14,23,10,10,12,...,3.988000e-07,5.047000e-07,8.090000e-07,5.506000e-07,6.371000e-07,1.132100e-06,8.583000e-07,6.703000e-07,False,"reward, prize"
6715,prouentus,168,353,164,274,330,141,508,78,201,...,2.026020e-05,3.936500e-06,1.355050e-05,5.016800e-06,2.201400e-05,4.528400e-06,2.861100e-06,9.848100e-06,True,Wind
5916,lucubratio,200,175,146,188,430,328,296,293,241,...,1.180520e-05,1.478700e-05,1.624710e-05,1.517290e-05,1.776690e-05,3.234560e-05,6.008200e-06,1.325640e-05,True,"night work, work-by-nightlamp, nocturnal study..."
2148,ciuilis,630,963,884,1306,1441,2339,1170,1773,1462,...,4.666230e-05,8.947870e-05,9.856130e-05,8.026940e-05,1.136092e-04,1.602726e-04,6.008200e-06,6.811950e-05,True,


In [86]:
subcorpus_counts_df.head(5)

4349

In [87]:
filtered_vocab_df.head(20)

3950

In [19]:
filtered_vocab_df.to_json("../data/filtered_vocab_df.json")
filtered_vocab_df.to_json("/srv/webserver/apps/iweems_app/data/filtered_vocab_df.json")
filtered_vocab_df.to_json("~/notebooks/iweems_huggingface/data/filtered_vocab_df.json")

In [18]:
!cp ../data/filtered_vocab_df.json /srv/data/tome/noscemus/filtered_vocab_df.json

In [90]:
resp_json[0]["translations_unstructured"]["en"]

'horse'

In [91]:
def simple_translation(target):
    try:
        base_url = "https://www.latin-is-simple.com/api/vocabulary/search/?query={}&forms_only=true"
        resp_json = requests.get(base_url.format(target)).json()
        transl = resp_json[0]["translations_unstructured"]["en"]
        if transl == "still in translation":
            transl = ""
        return transl
    except:
        return ""

In [ ]:
%%time
filtered_vocab_df["transl"] = filtered_vocab_df["word"].apply(simple_translation)

In [46]:
filtered_vocab_df.head(20)

,word,1501-1550,1551-1600,1601-1650,1651-1700,mean,in_lila_embeddings,in_lasla,in_operamaiora,transl
986,dico,43255,122549,42457,72224,70121.25,True,True,True,"say, call, tell"
11948,pars,33225,93059,44337,52385,55751.50,True,True,True,part
7039,possum,25029,82700,38391,76841,55740.25,True,True,True,"be able, can"
10696,habeo,31690,89199,39443,59770,55025.50,True,True,True,"have, hold, possess, consider, think"
141,facio,31814,98524,36944,51438,54680.00,True,True,True,"do, make, handle"
1007,liber,39559,65900,38496,53100,49263.75,True,True,True,"book, volume, inner bark of a tree, book, book"
4926,res,29069,70669,28291,51565,44898.50,True,True,True,suddenly
4477,uideo,21536,59099,26372,50503,39377.50,True,True,True,
11185,locus,23102,56782,31421,43680,38746.25,True,True,True,"place, location"
878,corpus,16979,63171,22871,31938,33739.75,True,True,True,"body, corpus"


In [75]:
filtered_vocab_df.to_json("../data/filtered_vocab_df.json")

In [2]:
!cp ../data/filtered_vocab_df.json /srv/data/tome/noscemus/filtered_vocab_df.json